In [27]:
pip install opencv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement opencv2 (from versions: none)
ERROR: No matching distribution found for opencv2


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import os



In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,ML_PROJECT/images\N1.xml,1093,1396,645,727
1,ML_PROJECT/images\N100.xml,134,301,312,350
2,ML_PROJECT/images\N101.xml,31,139,128,161
3,ML_PROJECT/images\N102.xml,164,316,216,243
4,ML_PROJECT/images\N103.xml,813,1067,665,724


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][10]
filename

'ML_PROJECT/images\\N109.xml'

In [5]:
cd

C:\Users\HP


In [6]:
import os

os.chdir('C:/Users/HP/ML_PROJECT')  

print(os.getcwd()) 


C:\Users\HP\ML_PROJECT


In [7]:
cd

C:\Users\HP


In [8]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('ML_PROJECT/images',filename_image)
    return filepath_image

In [9]:
getFilename(filename)

'ML_PROJECT/images\\N109.jpeg'

In [10]:
image_path = list(df['filepath'].apply(getFilename))
image_path 

['ML_PROJECT/images\\N1.jpeg',
 'ML_PROJECT/images\\N100.jpeg',
 'ML_PROJECT/images\\N101.jpeg',
 'ML_PROJECT/images\\N102.jpeg',
 'ML_PROJECT/images\\N103.jpeg',
 'ML_PROJECT/images\\N104.jpeg',
 'ML_PROJECT/images\\N105.jpeg',
 'ML_PROJECT/images\\N106.jpeg',
 'ML_PROJECT/images\\N107.jpeg',
 'ML_PROJECT/images\\N108.jpeg',
 'ML_PROJECT/images\\N109.jpeg',
 'ML_PROJECT/images\\N11.jpeg',
 'ML_PROJECT/images\\N110.jpeg',
 'ML_PROJECT/images\\N111.jpeg',
 'ML_PROJECT/images\\N112.jpeg',
 'ML_PROJECT/images\\N113.jpeg',
 'ML_PROJECT/images\\N114.jpeg',
 'ML_PROJECT/images\\N116.jpeg',
 'ML_PROJECT/images\\N117.jpeg',
 'ML_PROJECT/images\\N118.jpeg',
 'ML_PROJECT/images\\N119.jpeg',
 'ML_PROJECT/images\\N12.jpeg',
 'ML_PROJECT/images\\N120.jpeg',
 'ML_PROJECT/images\\N121.jpeg',
 'ML_PROJECT/images\\N122.jpeg',
 'ML_PROJECT/images\\N123.jpeg',
 'ML_PROJECT/images\\N124.jpeg',
 'ML_PROJECT/images\\N126.jpeg',
 'ML_PROJECT/images\\N127.jpeg',
 'ML_PROJECT/images\\N128.jpeg',
 'ML_PROJECT/i

In [11]:
file_path = image_path[0]
file_path 

'ML_PROJECT/images\\N1.jpeg'

In [12]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [16]:


from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [17]:
labels = df.iloc[:,1:].values

In [18]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [19]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [20]:
X.shape,y.shape

((225, 224, 224, 3), (225, 4))

In [21]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

In [22]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [23]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [24]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 73,663,490 (281.00 MB)

 Trainable params: 19,326,754 (73.73 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

## model Tarining

In [26]:
from tensorflow.keras.callbacks import TensorBoard

In [27]:
tfb = TensorBoard('object_detection')

In [28]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - loss: 0.1303 - val_loss: 0.1154
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 0.1055 - val_loss: 0.1118
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 0.0944 - val_loss: 0.0912
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 0.0850 - val_loss: 0.0910
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - loss: 0.0868 - val_loss: 0.0948
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - loss: 0.0834 - val_loss: 0.0942
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - loss: 0.0874 - val_loss: 0.0898
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - loss: 0.0813 - val_loss: 0.0892
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 0.0820 - val_loss: 0.0881
Epoch 10/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 0.0779 - val_loss: 0.0845
Epoch 11/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 0.0796 - val_loss: 0.0879
Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 0.0

In [36]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

Epoch 102/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 9.8205e-04 - val_loss: 0.0059
Epoch 103/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - loss: 7.6372e-04 - val_loss: 0.0052
Epoch 104/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 5.3862e-04 - val_loss: 0.0054
Epoch 105/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 8.3392e-04 - val_loss: 0.0060
Epoch 106/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 0.0011 - val_loss: 0.0067
Epoch 107/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 7.3858e-04 - val_loss: 0.0056
Epoch 108/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 6.1312e-04 - val_loss: 0.0057
Epoch 109/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 5.7967e-04 - val_loss: 0.0055
Epoch 110/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 4.6676e-04 - val_loss: 0.0056
Epoch 111/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - loss: 6.5911e-04 - val_loss: 0.0060
Epoch 112/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 4.5207e-04 - val_loss: 0.0059
Epo

In [37]:
model.save('ML_PROJECT/models/object_detection1.keras')

In [31]:
predicted_labels = model.predict(x_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 27s 12s/step


In [32]:
true_labels = y_test
